In [1]:
# Imports the Google Cloud client library
from google.cloud import language_v2
import pandas as pd
import numpy as np

# Instantiates a client
client = language_v2.LanguageServiceClient()

In [2]:
# Read CSV file
survey = pd.read_csv("./data/survey_clean_stemmed.csv")
survey.head()

,created_at,user_id,topic_id,stemmed_message
0,2023-10-24,1,0,cerita rasa tugas dimensi pesimis tugas sanggu...
1,2023-10-24,2,1,senang ajar materi 3d materi tarik manfaat suk...
2,2023-10-24,2,0,senang ajar materi 3d materi tarik manfaat suk...
3,2023-10-24,3,0,ajar materi seru awal susah basic web damping ...
4,2023-10-24,4,1,pusing ajar blender ya allah bismillah lancar ...


In [3]:
# Iterate over each row and do sentiment analysis
for index, row in survey.iterrows():
    try:
        text = row["stemmed_message"]
        document = language_v2.types.Document(
            content=text,
            type_=language_v2.types.Document.Type.PLAIN_TEXT,
            language_code="id",
        )
        sentiment = client.analyze_sentiment(
            request={"document": document}
        ).document_sentiment
        survey.loc[index, "sentiment_score"] = sentiment.score
        survey.loc[index, "sentiment_magnitude"] = sentiment.magnitude

        # add new column for sentiment label, above 0 is positive, below 0 is negative
        if sentiment.score > 0:
            survey.loc[index, "sentiment_label"] = "positive"
        else:
            survey.loc[index, "sentiment_label"] = "negative"
    except Exception as e:
        print(f"Error at index {index}: {e}")
        continue

Error at index 319: 500 Internal error encountered.
Error at index 426: 500 Internal error encountered.
Error at index 430: 500 Internal error encountered.
Error at index 544: 500 Internal error encountered.


In [4]:
import os

# Define the file path
file_path = "./data/survey_clean_stemmed_sentiment_googleNLP.csv"

# Check if the directory exists
directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# Try to write the file
try:
    survey.to_csv(file_path, index=False)
    print(f"File saved successfully to {file_path}")
except Exception as e:
    print(f"Failed to write file to {file_path}. Error: {e}")

File saved successfully to ./data/survey_clean_stemmed_sentiment_googleNLP.csv
